In [4]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import tflearn as tfl
import random


In [23]:
label = str(random.randint(0,100000000))
data = pd.read_csv('combined.csv',index_col=0)

In [24]:
del data['precipType']
X = data.iloc[:,0:15]
X.replace(np.nan,value = 0,inplace=True)
Y = data.iloc[:,-1:]
X


,apmaxTemp,apminTemp,cloudCover,dewPoint,humidity,moonPhase,precipIntensity,precipIntensityMax,precipProb,pressure,tempMax,tempMin,visibility,windBearing,windSpeed
7929,76.59,54.74,0.13,53.60,0.66,0.99,0.0000,0.0000,0.00,1024.55,76.59,54.74,9.80,115,6.25
6147,41.51,28.37,0.32,29.25,0.73,0.78,0.0015,0.0048,0.20,1010.31,41.51,33.11,7.85,343,2.39
331,77.66,66.54,0.05,65.77,0.82,0.40,0.0000,0.0000,0.00,1016.21,77.66,66.54,9.78,33,1.46
9150,90.60,66.29,0.01,43.88,0.29,0.77,0.0000,0.0000,0.00,1010.87,95.06,66.29,9.79,255,5.98
3187,80.16,55.24,0.04,44.63,0.44,0.95,0.0000,0.0000,0.00,1012.86,81.61,55.24,9.85,74,1.48
4909,32.66,15.27,0.41,23.25,0.86,0.87,0.0017,0.0055,0.24,1030.83,32.66,21.84,7.93,135,2.24
147,84.40,51.64,0.02,24.25,0.20,0.61,0.0000,0.0000,0.00,1014.85,89.36,51.64,10.00,323,2.18
9147,82.00,51.79,0.09,57.37,0.75,0.62,0.0000,0.0000,0.00,1015.31,81.67,51.79,8.26,164,7.35
6250,80.90,56.89,0.00,31.21,0.25,0.66,0.0000,0.0000,0.00,1010.93,83.91,56.89,9.94,304,6.28
8652,85.52,61.39,0.04,43.45,0.35,0.47,0.0000,0.0000,0.00,1012.05,89.24,61.39,9.75,296,3.34


In [25]:
enc = sklearn.preprocessing.OneHotEncoder(sparse = False)
enc.fit(Y)
Y = enc.transform(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = .2)

In [26]:
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = .5)

In [28]:
test_net = tfl.input_data(shape = [None,15],name = 'input')
test_net = tfl.layers.normalization.batch_normalization(test_net)
#                                                                                                                                   Regularizer helps overfitting
test_net = tfl.layers.core.fully_connected(test_net,2048,activation='relu',weights_init = 'normal',bias = True,bias_init = 'normal',regularizer = 'L2',name = 'FC1')
#Dropout Layer helps overfitting
test_net = tfl.layers.core.dropout(test_net,.73, name = 'DO1')
test_net = tfl.layers.core.fully_connected(test_net,1024,activation='relu',weights_init = 'normal',bias = True,bias_init = 'normal',regularizer = 'L2', name = 'FC2')
test_net = tfl.layers.core.dropout(test_net,.73, name = 'DO2')
test_net = tfl.layers.core.fully_connected(test_net,2,activation='relu',bias = True,name = 'To_Output')
#                                           This learning rate seems to marginally help with local minimums
test_net = tfl.layers.estimator.regression(test_net, name = "output",learning_rate = .0009)

# Problems with overfitting at .45ish and local minimum at .56ish, If succesful, should get model with 99% accuracy +- .5%
model = tfl.DNN(test_net, tensorboard_verbose = 0)
model.fit(np.array(X_train) , y_train ,validation_set = (np.array(X_val), y_val),n_epoch = 40, snapshot_step = 5000, show_metric = True,run_id = 'run_big' + label)
#Best Accuracy, 99%+-.5%

IndexError: list index out of range

In [21]:
X_val.shape

(1883, 14)